## Getting started

thermodiff is not uploaded to PyPI. To install it you can do:

```shell
pip install git+https://github.com/SalvadorBrandolin/thermodiff
```

Obtaining derivatives of thermodynamic expressions is an important but not
trivial task. Derivatives of Residual Helmholtz free energy and Excess Gibbs
free energy models are used to evaluate of the thermodynamic properties of
mixtures, such as enthalpy, entropy, and pressure.

Moreover, sometimes it's required to obtain derivatives of expressions with 
thermodynamic-involved magnitudes/variables for the development of algorithms,
for example, for phase envelope calculations.

## Tutorial: Let's get the derivatives of the UNIQUAC model

The UNIQUAC (**uni**versal **qua**si**c**hemical) Excess Gibbs free energy 
model is given by:

$$ 
\frac{G^E}{RT} = \sum_k n_k \ln\frac{\phi_k}{x_k}
+ \frac{z}{2}\sum_k q_k n_k \ln\frac{\theta_k}{\phi_k}
- \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right)
$$

With:

$$x_k = \frac{n_k}{\sum_l n_l}$$

$$ \phi_k = \frac{r_k n_k}{\sum_l r_l n_l} $$

$$ \theta_k = \frac{q_k n_k}{\sum_l q_l n_l} $$

$$ \tau_{lk} = \exp \left[\frac{-\Delta U_{lk}}{R T} \right] $$

$$
\frac{-\Delta U_{lk}}{R T} = a_{lk}+\frac{b_{lk}}{T}+c_{lk}\ln T + d_{lk}T +
e_{lk}{T^2}
$$

A simplification could be made to avoid division by zero, replacing $\phi_k$
and $x_k$ on the Gibbs free energy. The result is:

$$ 
\frac{G^E}{RT} = \sum_k n_k \ln \left(\frac{n_T r_k}{\sum_l r_l n_l} \right)
+ \frac{z}{2}\sum_k q_k n_k \ln \left(\frac{q_k \sum_l r_l n_l}{r_k \sum_l q_l
n_l} \right) - \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right) 
$$

Or directly how we will use it in this tutorial:

$$ 
G^E = RT\left(\sum_k n_k \ln \left(\frac{n_T r_k}{\sum_l r_l n_l} \right)
+ \frac{z}{2}\sum_k q_k n_k \ln \left(\frac{q_k \sum_l r_l n_l}{r_k \sum_l q_l
n_l} \right) - \sum_k q_k n_k \ln\left(\sum_l \theta_l \tau_{lk} \right)\right)
$$

First thing to note is that in the expression above, all the subscripts are $k$
or $l$. This is for a personal preference. I like to differentiate the
expression respect to $n_i$ and $n_j$. If we built our expression to
differentiate using $i$ and $j$ subscripts, we wouldn't be able to
differentiate correctly.


`thermodiff` already provide instantiated SymPy. Please use them since
internally when the $\frac{\partial}{\partial T}$ is performed, it will use
those instances.



In [2]:
from thermodiff import n, P, V, T  # mole number vector, pressure, volume, temperature
from thermodiff import k, l, m     # indexes
from thermodiff import R           # universal gas constant

import thermodiff as td

import sympy as sp # we still could need sympy

To use `thermodiff`, you should imagine that later you will have to implement
those equations by yourself. So, differentiate as you program, by functions.
Even a very recommended practice is to differentiate each term individually.
This avoids that SymPy heavily simplifies the expression to more non-readable
forms.

Let's start by the easiest but instructive part, $\tau_{lk}$. We have to
define the parameters $a_{lk}$, $b_{lk}$, $c_{lk}$, $d_{lk}$ and $e_{lk}$:

In [4]:
# The parameters of the model:
a_lk = sp.Symbol("a_{lk}")
b_lk = sp.Symbol("b_{lk}")
c_lk = sp.Symbol("c_{lk}")
d_lk = sp.Symbol("d_{lk}")
e_lk = sp.Symbol("e_{lk}")

# The SymPy expression
tau_lk_expr = sp.exp(a_lk + b_lk/T + c_lk*sp.ln(T) + d_lk*T + e_lk*T**2)

tau_lk_expr

exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)

Now we use the `thermodiff` class:

In [5]:
tau_lk_diffs = td.DiffPlz(tau_lk_expr)

Inside the attributes we have the derivatives respect to $T$:

In [6]:
tau_lk_diffs.dt

(2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)

In [7]:
tau_lk_diffs.dt2

(2*e_{lk} - c_{lk}/T**2 + 2*b_{lk}/T**3)*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T) + (2*T*e_{lk} + d_{lk} + c_{lk}/T - b_{lk}/T**2)**2*exp(T**2*e_{lk} + T*d_{lk} + a_{lk} + c_{lk}*log(T) + b_{lk}/T)